In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [3]:
URLs= pd.read_json('image_data.json')
anime = pd.read_csv('anime.csv')
Synopsis = pd.read_csv('synopsis.csv')


In [4]:
Synopsis.rename(columns={'sypnopsis':'synopsis'}, inplace=True)

In [5]:
anime.rename(columns={'MAL_ID':'id','English name':'name'}, inplace=True)
c_anime=anime.drop(columns =['Rating', 'Ranked', 'Popularity', 'Members',
       'Favorites', 'Watching', 'Completed', 'On-Hold', 'Dropped',
       'Plan to Watch', 'Score-10', 'Score-9', 'Score-8', 'Score-7', 'Score-6',
       'Score-5', 'Score-4', 'Score-3', 'Score-2', 'Score-1','Premiered','Licensors','Japanese name','name','Score','Episodes'
       ,'Producers'])


In [6]:

c_anime.replace('Unknown', np.nan, inplace=True)
c_anime = c_anime.dropna()

In [7]:
c_anime.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 8733 entries, 0 to 17552
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        8733 non-null   int64 
 1   Name      8733 non-null   object
 2   Genres    8733 non-null   object
 3   Type      8733 non-null   object
 4   Aired     8733 non-null   object
 5   Studios   8733 non-null   object
 6   Source    8733 non-null   object
 7   Duration  8733 non-null   object
dtypes: int64(1), object(7)
memory usage: 614.0+ KB


In [8]:
c_anime['Genres']

0          Action, Adventure, Comedy, Drama, Sci-Fi, Space
1                    Action, Drama, Mystery, Sci-Fi, Space
2        Action, Sci-Fi, Adventure, Comedy, Drama, Shounen
3        Action, Mystery, Police, Supernatural, Drama, ...
4                Adventure, Fantasy, Shounen, Supernatural
                               ...                        
17524                                Martial Arts, Fantasy
17526                                         Music, Drama
17532               Action, Martial Arts, Fantasy, Shounen
17538                                       Comedy, Parody
17552                                       Comedy, Sports
Name: Genres, Length: 8733, dtype: object

In [9]:
#  convert words of string to list of words
def convert_to_list(x):
    if isinstance(x, list):
        return x
    else:
        return x.split(',') 



def convert_to_list1(x):
    if isinstance(str(x), list):
        return x
    else:
        return str(x).split(' ') 


# to remove empty spaces between wrods in string
def remove_empty_spaces(x):
    return x.strip().replace(' ', '')


cv= CountVectorizer(max_features=5000,stop_words='english')

def stem(x):
    y=[]
    for i in x.split():
        y.append(PorterStemmer().stem(i))
    return " ".join(y)


In [10]:
c_anime["Genres"]=c_anime['Genres'].apply(convert_to_list)

In [11]:

# merge a_anime with Synopsis on id left and MAL_ID right
c_anime_synopsis = pd.merge(c_anime, Synopsis, left_on='id', right_on='MAL_ID')

In [12]:
c_anime_synopsis.drop(columns=['MAL_ID','Name_y','Score','Genres_y','Duration','Aired'], inplace=True)


In [13]:
c_anime_synopsis['Studios'] =c_anime_synopsis['Studios'].apply(remove_empty_spaces)
c_anime_synopsis['Type'] = c_anime_synopsis['Type'].apply(remove_empty_spaces)
c_anime_synopsis['Source'] = c_anime_synopsis['Source'].apply(remove_empty_spaces)

In [14]:
c_anime_synopsis['synopsis'] = c_anime_synopsis['synopsis'].apply(convert_to_list1)
c_anime_synopsis['Type'] = c_anime_synopsis['Type'].apply(convert_to_list1)
c_anime_synopsis['Studios'] = c_anime_synopsis['Studios'].apply(convert_to_list1)
c_anime_synopsis['Source'] = c_anime_synopsis['Source'].apply(convert_to_list1)

In [15]:
c_anime_synopsis['Tags']=c_anime_synopsis['Genres_x']+c_anime_synopsis['Studios']+c_anime_synopsis['Type']+c_anime_synopsis['Source']+c_anime_synopsis['synopsis']


In [16]:
c_anime_synopsis.drop(columns=['Genres_x','Studios','Type','Source','synopsis'], inplace=True)
content=c_anime_synopsis

In [17]:
content.rename(columns={'Name_x':'name'}, inplace=True)

In [18]:
content['Tags']=content['Tags'].apply(lambda x: [i.lower() for i in x])
content['Tags'] = content['Tags'].apply(lambda x:" ".join(x))
# content['Tags'].apply(stem)

In [19]:
content['Tags'] = content['Tags'].apply(stem)

In [20]:
vectors = cv.fit_transform(content['Tags']).toarray()

In [21]:
vectors.shape

(8323, 5000)

In [22]:
cv.get_feature_names()

C:\Users\Dell\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['000',
 '10',
 '100',
 '10th',
 '11',
 '11th',
 '12',
 '12th',
 '13',
 '13th',
 '14',
 '14th',
 '15',
 '15th',
 '16',
 '16th',
 '17',
 '17th',
 '18',
 '18th',
 '19',
 '1999',
 '19th',
 '1pictur',
 '1pictures',
 '1st',
 '20',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '20th',
 '21',
 '21st',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '2nd',
 '30',
 '300',
 '3d',
 '3rd',
 '4th',
 '50',
 '500',
 '5th',
 '60',
 '6th',
 '7th',
 '8bit',
 '8th',
 '90',
 '9th',
 'abandon',
 'abduct',
 'abil',
 'abilities',
 'ability',
 'abl',
 'abnorm',
 'aboard',
 'abov',
 'abroad',
 'abrupt',
 'abruptli',
 'absence',
 'absolut',
 'absorb',
 'absurd',
 'abund',
 'abus',
 'abyss',
 'academ',
 'academi',
 'academy',
 'acceler',
 'accept',
 'access',
 'accid',
 'accident',
 'acclaim',
 'accompani',
 'accomplish',
 'accord',
 'account',
 'accus',
 'accustom',
 'ace',
 'achiev',
 'acknowledg',
 'acquaint',
 'acquaintances',
 'acquir',
 'act',
 'acta'

In [26]:
distance=cosine_similarity(vectors)

In [28]:
distance.shape

(8323, 8323)

In [38]:
content['name']=content['name'].apply(lambda x: x.lower())

In [39]:
content.head(1)

,id,name,Tags
0,1,cowboy bebop,action adventur comedi drama sci-fi space sunr...


In [74]:
def content_recomendor(anime_name):
        try:
                anime_name=anime_name.lower()
                # getting the index of the entered anime name
                given_index = content['name'].values.tolist().index(anime_name)    
                # getting its distances from other animes
                distances=distance[given_index]
                # # enumarating to make sure original index is preserved
                distances=list(enumerate(distances))
                # sorting the distance in revers order to get the closest animes and taking first 5 values also skipping the first 
                # one since it is the given anime
                recomended_anime=sorted(distances , key=lambda x:x[1] , reverse=True)[1:11]
                for i in recomended_anime:
                        print(content.iloc[i[0]]['name'])
        except:
                print("Anime not found")
        
    

In [76]:
content_recomendor('hunter x hunter')

hunter x hunter (2011)
hunter x hunter movie 2: the last mission
vampire hunter d (2000)
bounty hunter: the hard
city hunter: ai to shukumei no magnum
city hunter
toriko: jump super anime tour 2009 special
hunter x hunter movie 1: phantom rouge
one piece: adventure of nebulandia
one piece: episode of nami - koukaishi no namida to nakama no kizuna
